<a href="https://colab.research.google.com/github/MykhailoFokin/MachineLearning/blob/Dev/simple_classification_with_MSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import fnmatch
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score

from sklearn.model_selection import train_test_split

In [2]:
#Copy images from repo to colab
!git clone https://github.com/SilvesterHsu/ORLFaceRecognition-PCA.git

Cloning into 'ORLFaceRecognition-PCA'...
remote: Enumerating objects: 604, done.
remote: Total 604 (delta 0), reused 0 (delta 0), pack-reused 604
Receiving objects: 100% (604/604), 3.59 MiB | 5.47 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [144]:
dataset = []
indexes = []
test_matrix = []
train_matrix = []
test_labels = []
train_labels = []
dataset_shuffle = False
stratify = None
random_state = 42
path_x=[]

# read folder structure with files and put it to collection
folder = []
for i in os.walk(os.path.join('ORLFaceRecognition-PCA','att_faces')):
  folder.append(i)

for address, dirs, files in folder:
  for file in fnmatch.filter(files, '*.pgm'):
    path_x.append(os.path.join(address,file))

for file in sorted(path_x, key=lambda x: int(x.split("/",3)[3].rstrip('.pgm')+x.split("/",3)[2].lstrip("s").zfill(2))):

  img = cv2.imread(file)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
  gray.flatten().reshape(1, 10304)
  person = file.split("/",3)[2].lstrip("s")

  dataset.append(gray.flatten().reshape(1, 10304))
  indexes.append(int(person))

for split_mark in range(1, 19):

  if split_mark>=10:
    dataset_shuffle = True
    split_k=split_mark-9
  else:
    split_k=split_mark

  train_matrix, test_matrix, train_labels, test_labels = train_test_split(dataset, indexes, test_size=0.1*split_k, random_state=random_state, shuffle=dataset_shuffle, stratify=stratify)

  estimations_mse = []
  estimations_mae = []
  estimations_msle = []
  estimations_std = []

  for test_image in test_matrix:

    matrix_mse = []
    matrix_mae = []
    matrix_msle = []
    matrix_std = []

    for train_image in train_matrix:

      matrix_mse.append(mean_squared_error(test_image, train_image))
      matrix_mae.append(mean_absolute_error(test_image, train_image))
      matrix_msle.append(mean_squared_log_error(test_image, train_image))
      matrix_std.append(np.sqrt(mean_squared_error(test_image, train_image)))

    estimations_mse.append(train_labels[np.argmin(matrix_mse)])
    estimations_mae.append(train_labels[np.argmin(matrix_mae)])
    estimations_msle.append(train_labels[np.argmin(matrix_msle)])
    estimations_std.append(train_labels[np.argmin(matrix_std)])

  result_mse = np.equal(test_labels, estimations_mse)
  result_mae = np.equal(test_labels, estimations_mae)
  result_msle = np.equal(test_labels, estimations_msle)
  result_std = np.equal(test_labels, estimations_std)

  matrix_length = len(result_mse)

  evaluation_mse = np.sum(result_mse)/matrix_length
  evaluation_mae = np.sum(result_mae)/matrix_length
  evaluation_msle = np.sum(result_msle)/matrix_length
  evaluation_std = np.sum(result_std)/matrix_length

  print('MSE{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_mse))
  print('MAE{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_mae))
  print('MSLE{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_msle))
  print('STD{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_std))


MSE.  Train/Test : 90 / 10 - 1.000
MAE.  Train/Test : 90 / 10 - 0.075
MSLE.  Train/Test : 90 / 10 - 0.925
STD.  Train/Test : 90 / 10 - 1.000
MSE.  Train/Test : 80 / 20 - 1.000
MAE.  Train/Test : 80 / 20 - 0.075
MSLE.  Train/Test : 80 / 20 - 0.963
STD.  Train/Test : 80 / 20 - 1.000
MSE.  Train/Test : 70 / 30 - 1.000
MAE.  Train/Test : 70 / 30 - 0.066
MSLE.  Train/Test : 70 / 30 - 0.950
STD.  Train/Test : 70 / 30 - 1.000
MSE.  Train/Test : 60 / 40 - 1.000
MAE.  Train/Test : 60 / 40 - 0.081
MSLE.  Train/Test : 60 / 40 - 0.938
STD.  Train/Test : 60 / 40 - 1.000
MSE.  Train/Test : 50 / 50 - 0.930
MAE.  Train/Test : 50 / 50 - 0.080
MSLE.  Train/Test : 50 / 50 - 0.910
STD.  Train/Test : 50 / 50 - 0.930
MSE.  Train/Test : 40 / 60 - 0.846
MAE.  Train/Test : 40 / 60 - 0.062
MSLE.  Train/Test : 40 / 60 - 0.892
STD.  Train/Test : 40 / 60 - 0.846
MSE.  Train/Test : 30 / 70 - 0.814
MAE.  Train/Test : 30 / 70 - 0.071
MSLE.  Train/Test : 30 / 70 - 0.850
STD.  Train/Test : 30 / 70 - 0.814
MSE.  Train/T